## First, combine the real and fake dataframes into a single dataframe

In [1]:
import pickle
import pandas as pd
from pathlib import Path

with open(Path.cwd() / 'domain_df.pkl', 'rb') as f:
    domain_df = pickle.load(f)
with open(Path.cwd() / 'domain_dict.pkl', 'rb') as f:
    domain_dict = pickle.load(f)

NotImplementedError: cannot instantiate 'PosixPath' on your system

In [3]:
Path.cwd()

WindowsPath('c:/Users/ewais/Documents/GitHub/misinfo_detection/sydney_powell_analysis/mutual_hyperlinking')

In [47]:
import numpy as np
from tqdm import tqdm
from bs4 import BeautifulSoup
import os
import sys

def scrape_hyperlink_connections(source_dir, domain_df, domain_dict):
    '''
    Parses through the html in source dir, scraping outward referring links along the way.
    Returns a numpy matrix, connections, with link from site i to site j indicated by
    connections[i,j] = 1.

    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ ARGUMENTS ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ #
    - source_dir
        - source of HTML files, should be raw_sources/<real or fake>/html
    - domain_dict
        - dictionary of domain, index pairs
    '''
    connections = np.zeros(shape=(len(domain_dict), len(domain_dict)))
    skipped_folders = []
    total_connections = {}  # Keys are domain indices, values are full lists of domains connected

    for dir_ in tqdm([source_dir / s for s in os.listdir(source_dir)]):  # Folders corresponding to domains
        # Get index of domain to access while populating connections
        if current_dir == 'real':
            row = domain_df.loc[(domain_df['label'] == 'real')].loc[(domain_df['index']) == int(dir_.name)]
        else:
            row = domain_df.loc[(domain_df['label'] == 'fake')].loc[(domain_df['index']) == int(dir_.name)]
        
        try:  # some folders exist in sources that aren't in dataframe
            domain_idx = row['condensed_idx'].values[0]
        except:
            print(f'skipping folder {dir_}')
            skipped_folders.append(dir_)
            continue

        files = [dir_ / s for s in os.listdir(dir_)]  # HTML files
        links = []

        # Get links from files
        for html in files:
            # Open HTML
            with open(html, encoding='utf8') as f:
                soup = BeautifulSoup(f, 'html.parser')
            f.close()
            
            for link in soup.findAll('a', href=True):
                links.append(link['href'])
        
        # Distill links down to domains
        links = list(set(links))
        domains = [
                    tldextract.extract(l).subdomain + '.' +
                    tldextract.extract(l).domain + '.' +
                    tldextract.extract(l).suffix for l in links
        ]
        domains = list(set(domains))
        filtered_domains = []
        for d in domains:
            if 'www' in d:
                filtered_domains.append(d.replace('www', ''))
            else:
                filtered_domains.append(d)
        
        filtered_domains_no_subdomain = []
        unfiltered_domains = []
        # Populate connections
        for d in filtered_domains:
            unfiltered_domains.append(d)
            # If the domain is in the dataset keys outright
            if d in domain_dict.keys():
                connections[domain_idx, domain_dict[d]] = 1
            if not d.split('.')[-1]:
                continue
            # If there is a prefix to the domain and less than 4 domain elements
            # i.e. web.domain.co.uk would be 4, web.domain.com would be 3
            # Sometimes sites have multiple countries prepended, like de.vexels.com and br.vexels.com
            if d.split('.')[0] and len(d.split('.')) < 4:
                if '.' + d.split('.')[1] + d.split('.')[2] in domain_dict.keys():
                    connections[domain_idx, domain_dict[d]] = 1
                elif '.' + d.split('.')[0] + '.' + d.split('.')[2] in domain_dict.keys():
                    # connections[domain_idx, domain_dict[d]] = 1
                    filtered_domains_no_subdomain.append('.' + d.split('.')[0] + '.' + d.split('.')[2])
                else:
                    filtered_domains_no_subdomain.append('.' + d.split('.')[1] + '.' + d.split('.')[2])
            # If there is a prefix and the domain has 4 elements, i.e. web.domain.co.uk
            # We're only interested in selecting domain.co.uk
            elif d.split('.')[0] and len(d.split('.')) == 4:
                if '.' + d.split('.')[1] + '.' + d.split('.')[2] + '.' + d.split('.')[3] in domain_dict.keys():
                    connections[domain_idx, domain_dict[d]] = 1
                    filtered_domains_no_subdomain.append('.' + d.split('.')[1] + '.' + d.split('.')[2] + '.' + d.split('.')[3])
                elif '.' + d.split('.')[0] + d.split('.')[2] + d.split('.')[3] in domain_dict.keys():
                    connections[domain_idx, domain_dict[d]] = 1
                    filtered_domains_no_subdomain.append('.' + d.split('.')[0] + '.' + d.split('.')[2] + '.' + d.split('.')[3])
                else:
                    filtered_domains_no_subdomain.append('.' + d.split('.')[1] + '.' + d.split('.')[2] + '.' + d.split('.')[3])
            # If there is not a prefix and the domain has 3 elements, i.e. .facebook.com
            # Ignore empty lists
            elif d.split('.') != ['', '', ''] and len(d.split('.')) < 4:
                if '.' + d.split('.')[1] + '.' + d.split('.')[2] in domain_dict.keys():
                    connections[domain_idx, domain_dict[d]] = 1
                    filtered_domains_no_subdomain.append('.' + d.split('.')[1] + '.' + d.split('.')[2])
                else:
                    filtered_domains_no_subdomain.append(d)
            elif d.split('.') != ['', '', '', ''] and len(d.split('.')) == 4:
                if '.' + d.split('.')[1] + '.' + d.split('.')[2] + '.' + d.split('.')[3] in domain_dict.keys():
                    connections[domain_idx, domain_dict[d]] = 1
                    filtered_domains_no_subdomain.append('.' + d.split('.')[1] + '.' + d.split('.')[2] + '.' + d.split('.')[3])
                else:
                    filtered_domains_no_subdomain.append(d)


            filtered_domains_no_subdomain = list(set(filtered_domains_no_subdomain))
            total_connections[domain_idx] = filtered_domains_no_subdomain
    
    return connections, skipped_folders, total_connections, unfiltered_domains


real_dir = Path.cwd().parent / 'large_vm_analysis' / 'raw_sources' / 'real' / 'sources'
fake_dir = Path.cwd().parent / 'large_vm_analysis' / 'raw_sources' / 'fake' / 'sources'

domain_dict = {}
for _, row in df.iterrows():
    domain_dict[row.domain.replace('www', '')] = row['condensed_idx']
connections, skipped_folders, total_connections, unfiltered_domains = scrape_hyperlink_connections(real_dir, fake_dir, df, domain_dict)  # Accidentally had break statement in loop, never went to fake dir

print(total_connections)
print(np.sum(connections[3858,:]))
# plt.figure(figsize = (20,20))
# plt.imshow(connections)

  0%|          | 1/2491 [00:02<1:50:46,  2.67s/it]

domain_idx: 6959
['', 'feinstein', 'senate', 'gov']
['hls', 'harvard', 'edu']
found
condition 3: ['', 'dailysignal', 'com']
found
condition 3: ['', 'pandemic', 'news']
found
condition 3: ['', 'judicialwatch', 'org']
condition 3: ['', 'azcapitoltimes', 'com']
found
condition 3: ['', 'bloomberg', 'com']
found
condition 3: ['', 'amazon', 'com']
condition 3: ['', 'dominicantoday', 'com']
condition 3: ['', 'cancer', 'gov']
found
condition 3: ['', 'infowars', 'com']
found
condition 3: ['', 'wsj', 'com']
condition 3: ['', 'evilgoogle', 'news']
found
condition 3: ['', 'alternativenews', 'com']
['', 'judiciary', 'senate', 'gov']
found
condition 3: ['', 'euthanasia', 'news']
condition 3: ['', 'divinitynow', 'org']
found
condition 3: ['', 'terrorism', 'news']
condition 3: ['', 'msn', 'com']
condition 3: ['', 'webseed', 'com']
found
condition 3: ['', 'berkeley', 'edu']
found
condition 3: ['', 'abortions', 'news']
found
condition 3: ['', 'theblaze', 'com']
condition 3: ['', 'medicalfascism', 'news'

  0%|          | 2/2491 [00:06<2:14:22,  3.24s/it]

domain_idx: 6304
found
condition 3: ['', 'southwichitanews', 'com']
found
condition 3: ['', 'facebook', 'com']
condition 3: ['', 'networkadvertising', 'org']
condition 3: ['', 'sacredheartfrontenac', 'com']
found
condition 3: ['', 'jocotoday', 'com']
condition 3: ['', 'standrewindependence', 'com']
condition 3: ['', 'gao', 'gov']
condition 3: ['', 'aboutads', 'info']
found
condition 3: ['', 'jamanetwork', 'com']
condition 3: ['', 'catholicgorillas', 'org']
condition 3: ['', 'fredoniaks', 'com']
found
condition 3: ['', 'va', 'gov']
condition 3: ['', 'stpatricksparsons', 'org']
condition 3: ['', 'acellusacademy', 'com']
condition 3: ['', 'stfrancis-stambrose', 'org']
found
condition 3: ['', 'littleappletimes', 'com']
condition 3: ['', 'ollsmc', 'com']
found
condition 3: ['', 'northwichitanews', 'com']
condition 3: ['', 'wichitastandard', 'com']
found
condition 3: ['', 'nekansasnews', 'com']
found
condition 3: ['', 'wyandottetimes', 'com']
found
condition 3: ['', 'eckansasnews', 'com']
co

  0%|          | 2/2491 [00:14<5:09:51,  7.47s/it]

domain_idx: 7803
found
condition 3: ['', 'facebook', 'com']
['app', 'content', 'ad']
found
condition 3: ['', 'blogger', 'com']
['widgets', 'mgid', 'com']
found
condition 3: ['', 'political-discussion', 'com']
['blogger', 'googleusercontent', 'com']
['api', 'content', 'ad']
found
condition 3: ['', 'mgid', 'com']
['support', 'google', 'com']
['2', 'bp', 'blogspot', 'com']
['4', 'bp', 'blogspot', 'com']
condition 3: ['', 'content', 'ad']
['3', 'bp', 'blogspot', 'com']
['d32oduq093hvot', 'cloudfront', 'net']
found
condition 3: ['', 'herbeauty', 'co']
found
condition 3: ['', 'domain', 'com']
['1', 'bp', 'blogspot', 'com']
condition 3: ['', 'content', 'ad']
['policies', 'google', 'com']
condition 3: ['', 't', 'co']
found
condition 3: ['', 'twitter', 'com']
found
condition 3: ['', 'brainberries', 'co']
{6959: ['.azcapitoltimes.com', '.centerformedicalprogress.org', '.naturalnews.com', '.dominicantoday.com', '.vatican.va', '.eugenics.news', '.cruz.senate.gov', '.thegatewaypundit.com', '.divini

In [50]:
np.sum(connections[7803])

8.0

In [42]:
from itertools import permutations

def extract_unique_domains(unfiltered_domains):
    '''
    Takes a list of tldextract processed domains and returns a list of unique domains
    This will filter things like [.de.vexels.com, .br.vexels.com, .vexels.tapfield.com, .vexels.com]
    down to [.vexels.com]
    '''
    perms = []
    for d in unfiltered_domains:
        if len(d.split('.')) < 3:
            continue
        elif d.split('.')[0] and len(d.split('.')) == 3:
            for x in permutations([x for x in d.split('.')[:-1] if x]):
                perms.append(x)
    print(perms)

extract_unique_domains(unfiltered_domains)
print(total_connections)

[('images', 'vexels'), ('vexels', 'images'), ('br', 'vexels'), ('vexels', 'br'), ('de', 'vexels'), ('vexels', 'de'), ('home', 'bluesnap'), ('bluesnap', 'home'), ('vexels', 'tapfiliate'), ('tapfiliate', 'vexels'), ('mockups', 'vexels'), ('vexels', 'mockups'), ('uy', 'linkedin'), ('linkedin', 'uy'), ('logomaker', 'vexels'), ('vexels', 'logomaker'), ('es', 'vexels'), ('vexels', 'es')]
{3858: ['.paypal.com', '.youtube.com', '.instagram.com', '.facebook.com', '.cookiepedia.co.uk', '.vexels.tapfiliate', '.linkedin.com', '.pinterest.com', '.vexels.com', '.onetrust.com', '.twitter.com', '.bluesnap.com']}


In [ ]:
with open(Path.cwd() / 'full_analysis' / 'connections.pkl', 'wb') as f:
    pickle.dump(connections, f)

with open(Path.cwd() / 'full_analysis' / 'domain_dict.pkl', 'wb') as f:
    pickle.dump(domain_dict, f)

with open(Path.cwd() / 'full_analysis' / 'skipped_folders.pkl', 'wb') as f:
    pickle.dump(skipped_folders, f)

print(f'skipped {len(skipped_folders)} folders')

In [15]:
with open('domain_dict.pkl', 'rb') as f:
    dd = pickle.load(f)
print(dd)

{'.9to5mac.com': 0, '.wfae.org': 1, '.climatefeedback.org': 2, '.wdbj7.com': 3, '.keyc.com': 4, '.wsaz.com': 5, '.publicintegrity.org': 6, '.deadline.com': 7, '.lawandcrime.com': 8, '.martech.org': 9, '.thelily.com': 10, '.theverge.com': 11, '.eater.com': 12, '.americanmilitarynews.com': 13, '.healthcareitnews.com': 14, '.streetsblog.org': 15, '.digitaltrends.com': 16, '.techrepublic.com': 17, '.bridgemi.com': 18, '.military.com': 19, '.lincolncityhomepage.com': 20, '.oklahomawatch.org': 21, '.opensecrets.org': 22, '.dmagazine.com': 23, '.vpap.org': 24, '.arstechnica.com': 25, '.investopedia.com': 26, '.civileats.com': 27, '.taskandpurpose.com': 28, '.popular.info': 29, '.avclub.com': 30, '.jalopnik.com': 31, '.pappaspost.com': 32, '.tapinto.net': 33, '.therealnews.com': 34, '.scientificamerican.com': 35, '.courierpostonline.com': 36, '.vtdigger.org': 37, '.onmilwaukee.com': 38, '.marketwatch.com': 39, '.abajournal.com': 40, '.plantbasednews.org': 41, '.lbpost.com': 42, '.globalcitizen